## Linear Fit with Trial & Error
In this exercise we simulate a simple linear fit algorithm. The idea is to vary the slope and find the value that corresponds to the _least square_.

First, we create some fictional data. The y-values linearly depend on x, but with a random perturbation.

In [ ]:
import numpy as np

n_points = 10 # Number of data points
slope = 2 # Slope of the linear function
noise = 1 # Noise amplitude

rng = np.random.default_rng() # Random number generator
x = np.arange(n_points) # Generate x values
y = slope * x + noise * rng.random(n_points) # Generate linear data with noise

Make a graph of y vs. x.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.plot(x, y, '.')
ax.set_title('Linear Data with Noise')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.grid(True)
plt.show()

Now we assume that we do not know the "exact" slope, but we can estimate it to be between 1 and 2.

In [ ]:
def sq_dev(x, y, m):
    diff = y - m * x
    sq = diff ** 2
    return np.sum(sq)

m_min = 1
m_max = 3

step = 0.001
trials = np.arange(m_min, m_max, step)

lsq = np.array([sq_dev(x, y, t) for t in trials])

min_index = np.argmin(lsq)
opt = trials[min_index]

fig, ax = plt.subplots()
ax.plot(trials, lsq, '-')
ax.plot(opt, lsq[min_index], 'ro', label=f'least square deviation for m = {opt:.3f}')
ax.set_title('Square Deviation vs Slope')
ax.set_xlabel('slope')
ax.set_ylabel('square deviation')
ax.legend()
ax.grid(True)
plt.show()

Compare the result to the values obtained with a built-in fit function.

In [ ]:
from scipy.optimize import curve_fit


def f(x, m):
    return m * x

coeff, pcov = curve_fit(f, x, y)
m = coeff[0]

print(f'Best slope according to scipy.curve_fit: m = {m:.3f}')

As long as the steps for the guess are small enough, the result of the (more efficient) curve_fit can be reproduced.

## Formal derivation
Show that the value for the slope that minimises the square deviation is given by
$$ m = \frac{\sum_{i=1}^n x_i y_i}{\sum_{i=1}^n x_i^2} $$

For an arbitrary slope $m$ the square deviation is given by
$$ \chi^2 = \sum_{i=1}^n (m x_i - y_i)^2 $$

To minimise this expression, we have to take its derivative with respect to $m$:
$$ \frac{\textrm{d}\chi^2}{\textrm{d}m} = \sum_{i=1}^n 2 (m x_i - y_i) x_i = 
2 \left(m \sum_{i=1}^n x_i^2 - \sum_{i=1}^n x_i y_i \right) $$

For a minimum this has to be zero. It follows the expression above for the least square.

#### Numerical verification
Verify if the formal expression leads to the same result for the optimum slope as the calculations above.

In [ ]:
sxi2 = np.sum(x**2) # sum of x_i^2
sxiyi = np.sum(x*y) # sum of x_i y_i

m = sxiyi / sxi2

print(f'Best slope according to scipy.curve_fit: m = {m:.3f}')

## Error of slope
In order to calculate the error of the slope, we can use the general rule for the error propagation of a quantity that depends on the measured values $x_1, x_2, \dots, x_n$ and $y_1, y_2, \dots, y_n$:
$$ \Delta m = \sum_{i=1}^n \frac{\partial m}{\partial x_i} \Delta x_i + \sum_{i=1}^n \frac{\partial m}{\partial y_i} \Delta y_i $$

In the following we assume that the uncertainties for all $x$ values are the same, and that all uncertainties for $y$ are the same, i.e. $\Delta x_i = \Delta x$ and $\Delta y_i = \Delta y$ for all values of $i$.

Derive a formal expression for $\Delta m$ and calculate the numerical value for uncertainties $\Delta x = 0.1$ and $\Delta y = 0.5$.

The derivatives of $m$ with respect to $x_i$ are (with the product rule)
$$ \dfrac{\partial m}{\partial x_i} = \frac{\frac{\partial}{\partial x_i} \left( \sum_{j=1}^n x_j y_j \right)}{\sum_{j=1}^n x_j^2}
+ \sum_{j=1}^n x_j y_j \cdot {\frac{\partial}{\partial x_i} \left(\sum_{j=1}^n x_j^2 \right)^{-1}} $$

The first term is just 
$$ \frac{y_i}{\sum_{j=1}^n x_j^2} $$

(all other terms in the sum are constant with respect to $x_i$), and the second term can be written as (using the chain rule)
$$ \sum_{j=1}^n x_j y_j \cdot \left( - \left(\sum_{j=1}^n x_j^2 \right)^{-2} \cdot 2 x_i \right) $$

For the derivatives with respect to $y_i$, the numerator is constant, so we find
$$ \dfrac{\partial m}{\partial y_i} = \frac{x_i}{\sum_{j=1}^n x_j^2} $$

Putting everything together, we find

$$ \Delta m = \left( \frac{\sum y_i}{\sum x_i^2} - 2 \frac{\sum x_i \sum x_i y_i}{\left( \sum x_i^2 \right)^2} \right) \Delta x + \frac{\sum x_i}{\sum x_i^2} \Delta y$$

#### Numerical calculation

In [ ]:
sxi = np.sum(x)
syi = np.sum(y)
sxi2 = np.sum(x**2)
sxiyi = np.sum(x*y)

dx = 0.1
dy = 0.5

dm = syi/sxi2 * dx + sxi/sxi2 * dy - 2*sxiyi * sxi / sxi2**2 * dx

print(f'{m:.3f} ± {dm:.3f}')